# Scaffold model of Cerebellum

## Cell positioning notebook

This place the main cells of cerebellar cortex  and deep cerebellar nuclei in a 3-dimensional volume according to their respective density distributions.    

First, install voxcellview library and some extensions

_Dependencies and output file_:

This notebook imports - and requires - two python files, namely:

1. **scaffold_params.py**, a configuration file
2. **scaffold_functions.py**, with the functions used in the notebook 

Both are stored in the *Network_files* directory of our storage.
They're not directly available from this notebook; in order to explore them, there are two main options:

1. download them from the storage
2. once the files have been downloaded in the workspace (3rd code cell in this notebook),  the user can create a new cell and open a file with the command _%load_ ***filename***, where the filename can be either *scaffold_params.py* or *scaffold_functions.py* . 

**NOTE for user**: if you want to modify some parameters/variables from *scaffold_params.py* or *scaffold_functions.py*, please consider that some changes may result in errors or unexpected results.
Parameters that are expected to be chosen by the user are displayed to the user in a cell below. Feel free to modify other parameters from the file but please keep track of their original values.   

This notebook will create a .hdf5 file and save it to *Network_file* folder of the storage. This final file is organized in five columns:
1. GID of neurons
2. Numerical ID for each cell type
3. x-position for each cell
4. y-position for each cell
5. z-position for each cell


## Setup of environment

In [ ]:
!pip install --quiet --upgrade "hbp-service-client==1.0.0"
!pip freeze | grep hbp

#### **<font color="red">If the version of the <i>hbp-service-client</i> is below 1.0.0 please restart the kernel and continue from the next Cell. Otherwise just continue without restarting the kernel...</font>**


Now downloading two .py files from the storage:
'scaffold_params.py' contains parameters to setup network building
'scaffold_functions.py' is a collection if functions that will be used for cells placement

In [ ]:
import os
client = get_hbp_service_client()
#client = get_bbp_client()
# Path to cerebellum collab storage
sp6_collab_path = '/8124'
bsp_folder = sp6_collab_path+'/BSP_usecases/Network_files/'

# path to local storage 
clb_path = get_collab_storage_path()
client.storage.download_file(bsp_folder+'scaffold_params.py', 'scaffold_params.py')
client.storage.download_file(bsp_folder+'scaffold_functions.py', 'scaffold_functions.py')

In [ ]:
from scaffold_params import *
from scaffold_functions import *
from ipywidgets import widgets

# Try to plot whole network with plotly
import plotly.plotly as py
import plotly.graph_objs as go 
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot
init_notebook_mode(connected=True)

This prints default values for 'save', 'plot' and 'volume_base_size' variables

In [ ]:
print(save, plot, volume_base_size)

## Choose some parameters for simulation

Here the user can set three parameters:
1. Save results: (*True*, *False*); choose True if want to save a .hdf5 file with positions in the collab. 
2. Plot results: (*True*, *False*)
3. Network size: choose the base size of the network; by now only three options available: **200, 300 or 400** micrometers

Use user defined variables to update network parameters

In [ ]:
save_on = True
plot_on = True
netsize = 400.

In [ ]:
save = save_on
plot = plot_on
volume_base_size[0], volume_base_size[1] = netsize, netsize

# Define pc and dcn values once volume base size has been defined
pc_in_volume = int(volume_base_size[0]*volume_base_size[1]*cells_density['purkinje'])
dcn_in_volume = int(pc_in_volume / pc_dcn_ratio)
cells_density['dcn'] = dcn_in_volume / (dcn_volume[0]*dcn_volume[1]*layers_thick['dcn'])

dcn_volume = volume_base_size / 2
filename = 'scaffold_full_dcn'
save_name = '{}_{}x{}'.format(filename, volume_base_size[0], volume_base_size[1])
save_name = save_name + '.hdf5'

print(save, plot, volume_base_size)

Import other required libraries

In [ ]:
import h5py
import matplotlib.pyplot as plt
import pandas as pd
from mpl_toolkits.mplot3d import Axes3D
import random
from scipy.spatial import distance
#from voxcellview.widget import VoxcellWidget

%matplotlib inline

## Cell Positioning:

Now meaningful variables are computed for each cell type; cells positioning is executed here.
This may take a while depending on network size.

In [ ]:
goc_in_volume, goc_eps, goc_sublayers, goc_height_placement, ngoc_per_sublayer, goc_bounds = sublayer_partitioning('granular', 'golgi', volume_base_size)
placement_stats['golgi']['total_n_golgi'] = goc_in_volume
placement_stats['golgi']['golgi_subl'] = ngoc_per_sublayer

v_occupied_goc = volume_occupied(goc_in_volume, cells_radius['golgi'])
glom_in_volume, glom_eps, glom_sublayers, glom_height_placement, nglom_per_sublayer, glom_bounds = sublayer_partitioning('granular', 'glomerulus', volume_base_size, v_occupied_goc)
placement_stats['glomerulus']['total_n_glomerulus'] = 	glom_in_volume
placement_stats['glomerulus']['glomerulus_subl'] = nglom_per_sublayer

v_occupied_glom = volume_occupied(glom_in_volume, cells_radius['glomerulus'])
grc_in_volume, grc_eps, grc_sublayers, grc_height_placement, ngrc_per_sublayer, grc_bounds = sublayer_partitioning('granular', 'granule', volume_base_size, v_occupied_goc, v_occupied_glom)
placement_stats['granule']['total_n_granule'] = 	grc_in_volume
placement_stats['granule']['granule_subl'] = ngrc_per_sublayer

sc_in_volume, sc_eps, sc_sublayers, sc_height_placement, nsc_per_sublayer, sc_bounds = sublayer_partitioning('molecular', 'stellate', volume_base_size)
bc_in_volume, bc_eps, bc_sublayers, bc_height_placement, nbc_per_sublayer, bc_bounds = sublayer_partitioning('molecular', 'basket', volume_base_size)
placement_stats['stellate']['total_n_stellate'] = sc_in_volume
placement_stats['stellate']['stellate_subl'] = nsc_per_sublayer
placement_stats['basket']['total_n_basket'] = bc_in_volume
placement_stats['basket']['basket_subl'] = nbc_per_sublayer


dcn_in_volume, dcn_eps, dcn_sublayers, dcn_height_placement, ndcn_per_sublayer, dcn_bounds = sublayer_partitioning('dcn', 'dcn', dcn_volume)
dcn_eps /= 10.   # Reducing the average dcn_eps
placement_stats['dcn']['total_n_dcn'] = dcn_in_volume
placement_stats['dcn']['dcn_subl'] = ndcn_per_sublayer


if goc_in_volume > 0.0:
    cells_placement(goc_sublayers, volume_base_size, 'golgi', goc_eps, goc_height_placement, ngoc_per_sublayer, goc_bounds)
if glom_in_volume > 0.0:
    cells_placement(glom_sublayers, volume_base_size,'glomerulus', glom_eps, glom_height_placement, nglom_per_sublayer, glom_bounds)
if grc_in_volume > 0.0:
    cells_placement(grc_sublayers, volume_base_size,'granule', grc_eps, grc_height_placement, ngrc_per_sublayer, grc_bounds)
if pc_in_volume > 0.0:
    purkinje_placement(pc_extension_dend_tree, pc_in_volume)
if sc_in_volume > 0.0:
    cells_placement(sc_sublayers, volume_base_size,'stellate', sc_eps, sc_height_placement, nsc_per_sublayer, sc_bounds)
if bc_in_volume > 0.0:
    cells_placement(bc_sublayers, volume_base_size,'basket', bc_eps, bc_height_placement, nbc_per_sublayer, bc_bounds)
if dcn_in_volume > 0.0:
    cells_placement(dcn_sublayers, dcn_volume, 'dcn', dcn_eps, dcn_height_placement, ndcn_per_sublayer, dcn_bounds)


In [ ]:
adapt_positions()
if ndcn_per_sublayer > 0.0:
    final_cell_positions['dcn'][:,[0,2]] = final_cell_positions['dcn'][:,[0,2]] + dcn_volume/2

# Data saving & plots

This cell is required for plotting and/or saving

In [ ]:
if save or plot:
    # Progressive indexing of neurons
    cel_num_vec = sorted(list(cell_type_ID.values()))
    
    cellID2type = {val: key for key, val in cell_type_ID.items()}
    max_shape_values = [final_cell_positions[cellID2type[val]].shape[0] for val in cel_num_vec]
    max_shape_values.insert(0,0)
    prog_nums = []
    for n in range(1,len(max_shape_values)):
        summed_prec = sum(max_shape_values[0:n])
        prog_nums.extend(range(summed_prec, summed_prec+max_shape_values[n]))
    
    data_matrix = np.empty((1, 4))

    for n in cel_num_vec:
        cell_id = np.zeros(final_cell_positions[cellID2type[n]].shape[0])+cell_type_ID[cellID2type[n]]
        positions = final_cell_positions[cellID2type[n]]
        pos_id = np.column_stack((cell_id, positions))
        data_matrix = np.concatenate((data_matrix, pos_id))

    data_matrix = data_matrix[1::]
    data_matrix = np.column_stack((prog_nums, data_matrix))
    
    # Convert data_matrix to a pandas DataFrame
    data_frame = pd.DataFrame(data_matrix, columns=['id', 'ntype', 'x', 'z', 'y'])
    data_frame['id'] += 1

    df_plot = data_frame[['ntype', 'x','y','z']]
    radius_vec = [cells_radius[cellID2type[x]] for x in data_frame['ntype']]

Code to save and plot cells positions.

The plot generated here is a plotly version of Fig.2 A of Casali et al (2019) *Reconstruction and Simulation of a Scaffold Model of the Cerebellar Network* Frontiers in Neuroinformatics,  doi.org/10.3389/fninf.2019.00037

Data are not exactly the same, because a fresh new scaffold has been generated in this notebook. 

In [ ]:
if save:
    # First, delete all scaffold*.hdf5 files from storage
    from fnmatch import fnmatch
    storage_files = client.storage.list(clb_path)
    competing_files = [files for files in storage_files if fnmatch(files, 'scaffold*.hdf5')]
    if len(competing_files) > 0:
        for files in competing_files:
            client.storage.delete(clb_path+'/'+files)
            
    # Create a .hdf5 file to store data; .hdf5 is the 'standard' format 
    f = h5py.File(save_name, 'w')
    f.create_dataset('positions', data=data_matrix)
    f.close()
    new_file = clb_path+'/'+save_name
        
    if client.storage.exists(new_file):
        client.storage.delete(new_file)
        
    client.storage.upload_file(save_name, new_file, 'text/plain')
    
    
if plot:
    mult = 2
    traceGrcs = go.Scatter3d(
                x = data_matrix[data_matrix[:,1]==cell_type_ID['granule'],2],
                y = data_matrix[data_matrix[:,1]==cell_type_ID['granule'],4],
                z = data_matrix[data_matrix[:,1]==cell_type_ID['granule'],3],
                name = 'granules',
                mode = 'markers',
                marker = dict(color='rgb(255, 0, 0)',
                              #sizemin=50,
                              sizemode='diameter',
                              size=np.repeat(cells_radius['granule']*mult, sum(data_matrix[:,1]==cell_type_ID['granule'])),
                             symbol='circle', 
                            #line=dict(
                            #    color='rgba(204, 204, 204)', width=0.5
                            #)
                              opacity=1.0)
                        )
    traceGloms = go.Scatter3d(
                x = data_matrix[data_matrix[:,1]==cell_type_ID['glomerulus'],2],
                y = data_matrix[data_matrix[:,1]==cell_type_ID['glomerulus'],4],
                z = data_matrix[data_matrix[:,1]==cell_type_ID['glomerulus'],3],
                name = 'glomeruli',
                mode = 'markers',
                marker = dict(color='rgb(0, 0, 255)',
                              sizemode='diameter',
                              size=np.repeat(cells_radius['glomerulus']*mult, sum(data_matrix[:,1]==cell_type_ID['glomerulus'])),
                             symbol='circle', 
                            #line=dict(
                            #    color='rgba(204, 204, 204)', width=0.5
                            #)
                              opacity=1.0)
                        )
    traceGocs = go.Scatter3d(
                x = data_matrix[data_matrix[:,1]==cell_type_ID['golgi'],2],
                y = data_matrix[data_matrix[:,1]==cell_type_ID['golgi'],4],
                z = data_matrix[data_matrix[:,1]==cell_type_ID['golgi'],3],
                name = 'Golgi cells',
                mode = 'markers',
                marker = dict(color='rgb(0, 255, 0)',
                              sizemode='diameter',
                              size=np.repeat(cells_radius['golgi']*mult, sum(data_matrix[:,1]==cell_type_ID['golgi'])),
                             symbol='circle', 
                            line=dict(
                                color='rgba(204, 204, 204)', width=0.000001
                            )
                            ,opacity=0.5)
                        )
    tracePCs = go.Scatter3d(
                x = data_matrix[data_matrix[:,1]==cell_type_ID['purkinje'],2],
                y = data_matrix[data_matrix[:,1]==cell_type_ID['purkinje'],4],
                z = data_matrix[data_matrix[:,1]==cell_type_ID['purkinje'],3],
                name = 'Purkinje cells',
                mode = 'markers',
                marker = dict(color='rgb(15, 83, 39)', 
                              sizemode='diameter',
                              size=np.repeat(cells_radius['purkinje']*mult, sum(data_matrix[:,1]==cell_type_ID['purkinje'])),
                             symbol='circle', 
                            line=dict(
                                color='rgba(204, 204, 204)', width=0.00000001
                            )
                            ,opacity=0.75)
                        )

    traceSCs = go.Scatter3d(
                x = data_matrix[data_matrix[:,1]==cell_type_ID['stellate'],2],
                y = data_matrix[data_matrix[:,1]==cell_type_ID['stellate'],4],
                z = data_matrix[data_matrix[:,1]==cell_type_ID['stellate'],3],
                name = 'stellate cells',
                mode = 'markers',
                marker = dict(color='rgb(224, 203, 24)', 
                              sizemode ='diameter',
                              size=np.repeat(cells_radius['stellate']*mult, sum(data_matrix[:,1]==cell_type_ID['stellate'])),
                             symbol='circle', 
                            line=dict(
                                color='rgba(204, 204, 204)', width=0.00000001
                            )
                            ,opacity=0.75)
                        )
    traceBCs = go.Scatter3d(
                x = data_matrix[data_matrix[:,1]==cell_type_ID['basket'],2],
                y = data_matrix[data_matrix[:,1]==cell_type_ID['basket'],4],
                z = data_matrix[data_matrix[:,1]==cell_type_ID['basket'],3],
                name = 'basket cells',
                mode = 'markers',
                marker = dict(color='rgb(129, 84, 102)',
                              sizemode='diameter',
                              size=np.repeat(cells_radius['basket']*mult, sum(data_matrix[:,1]==cell_type_ID['basket'])),
                             symbol='circle', 
                            line=dict(
                            color='rgba(204, 204, 204)', width=0.00000001
                            )
                            ,opacity=0.75)
                        )

    traceDCN = go.Scatter3d(
                x = data_matrix[data_matrix[:,1]==cell_type_ID['dcn'],2],
                y = data_matrix[data_matrix[:,1]==cell_type_ID['dcn'],4],
                z = data_matrix[data_matrix[:,1]==cell_type_ID['dcn'],3],
                name = 'DCN',
                mode = 'markers',
                marker = dict(color='rgb(128, 0, 0)', 
                              sizemode='diameter',
                              size=np.repeat(cells_radius['dcn']*mult, sum(data_matrix[:,1]==cell_type_ID['dcn'])),
                             symbol='circle', 
                            line=dict(
                                color='rgba(204, 204, 204)', width=0.00001
                            )
                            ,opacity=0.75)
                        )



    layout = go.Layout(
                scene = dict(
                    aspectmode='data')
                )
    data = [traceGrcs, traceGloms, traceGocs, tracePCs, traceSCs, traceBCs, traceDCN]
    fig = go.Figure(data=data, layout=layout)
    iplot(fig, filename='cell_placement')

Next Notebook to run: **Connectome**

In [ ]:
print(data_frame)